In [2]:
# Add directory above current directory to path
import sys; sys.path.insert(0, '..')

from src.chemkin_wrapper import chemkin_wrapper, plot_outputs
from src.chemkin import CHEMKIN 
from src.hychem import CONDITION, HYCHEM

%reload_ext autoreload
%autoreload 2

In [3]:
# d = chemkin_wrapper("../workingdir")
# d.head()
# plot_outputs(d)

In [4]:
ck = CHEMKIN()
cond = CONDITION(1300, 4, {'POSF10325': .004, 'AR': .996})
print(cond)
ck.chemkin_wrapper("../workingdir", cond).head()

Temperature: 1300 K
Pressure: 4 atm
POSF10325: 0.004 (fraction)
AR: 0.996 (fraction)



,t,POSF10325,C2H4,CH4,C3H6,iC4H8,C4H81,H2,C2H6,CO,C6H6,C2H2,C6H5CH3,C5H6,pC3H4,aC3H4,CH3,P,T
0,0.0000,0.004000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,4.0,1300.0
1,1.0069,0.003851,0.000247,0.000024,0.000116,0.000012,0.000037,0.000048,0.000002,0.0,0.000029,9.720700e-09,0.000027,4.192500e-11,4.426100e-09,1.880600e-09,0.000173,4.0,1296.3
2,2.0410,0.003698,0.000502,0.000060,0.000235,0.000025,0.000074,0.000109,0.000017,0.0,0.000059,7.462600e-08,0.000057,8.146200e-10,3.127900e-08,1.782000e-08,0.000313,4.0,1293.0
3,3.0143,0.003567,0.000720,0.000093,0.000336,0.000035,0.000105,0.000160,0.000046,0.0,0.000085,2.013600e-07,0.000081,3.649200e-09,7.886700e-08,5.397000e-08,0.000395,4.0,1290.4
4,4.1336,0.003431,0.000948,0.000130,0.000441,0.000046,0.000138,0.000213,0.000092,0.0,0.000113,4.214900e-07,0.000107,1.140300e-08,1.543800e-07,1.235900e-07,0.000445,4.0,1288.1
